In [35]:
from typing import Tuple

class math:

    """
    Zvlastni trida ve ktere budu definovat obecne prospesne veci, ktere maji co docineni s matikou. 
    Proc ne statismethod nebo inheritance metodami ? 
        -> pokud ji zavolam chci abych vedel ze pouziva  obecnou, nekde outside metodu nebo pokud by na tom pracovalo vic lidi 
            a nekoho napadlo zadefinovat si metodu se stejnym jmenem, ktera bude delat neco jinyho tak abych nemel override ale vedelo se ze se taha tato obecna metoda
    """

    def __init__(self):
        
        print("Thanks so much for using my very cool math class :) ")

    @classmethod
    def abs(self, num) -> int:

        return abs(num)
    @classmethod
    def sum(self, *args):

        val = 0
        
        for arg in args:

            val += arg

        return val

class Entity:

    def __init__(self, name: str, coords: Tuple[int, int], hit_points: int) -> None:
        self.name = name
        self.coords = coords
        self.hit_points = hit_points
        # "constructor", more precisely "initializer"
        # initialize Entity here
        # name is string such as "Rob"
        # coords is tuple to represent x and y coordinates in the field: (1, 2) menas x = 1 and y = 2; expect also negative values
        # hitpoints represents count of remaining hitpoints as int, 0 or less hitpoints means death
        
    def take_damage(self, damage_amount: int) -> None:
        # m# method to apply damage on this entity, it takes damage_amount and applies it by own rules to the entity
        # this is abstract method, just keep it as it is
        raise NotImplementedError("This method is abstract. Please implement it")  # in the child, not here :-) :-)

    def is_alive(self) -> bool:
        # method to check if entity is alive
        # return True if it has hit points above 0
        # return False if it has hit points equal to 0 or less
        if self.hit_points > 0:
            print('Ziju, sem v cajku')
            return True

    def get_distance(self, other_coords: Tuple[int, int]) -> int:
        # method to count distance to other object
        # for calculation of distance use Taxicab/Manhattan metric: https://en.wikipedia.org/wiki/Taxicab_geometry
        # returns the distance

        return math.sum(
            math.abs(self.coords[0] - other_coords[0]),
            math.abs(self.coords[1] - other_coords[1])
        )
            
class Rock(Entity):
    def __init__(self, name, coords, hit_points):
        super().__init__(name, coords, hit_points)

    def take_damage(self, damage_amount: int):                                                              #Nechci vratit neco o tom ze jako cislo?
        # method to apply damage on this entity
        # rock takes damage only when it is hit with damage_amount of at least 10,
        # each such hit takes him 1 hitpoint, smaller hits don't cause any damage to him
        if damage_amount >= 10:
            
            self.hit_points = self.hit_points - 1

            print(f' Auvi Auvi, ubral jsi mi jeden zivot :(, \n ted mam kvuli tobe {self.hit_points} zivotu')
            return self.hit_points
        else:
            print('Pohoda :)')
            return self.hit_points

class Furniture(Entity):

    def __init__(self, name, coords, hit_points):
        super().__init__(name, coords, hit_points)

    def take_damage(self, damage_amount: int) -> None:                                                              #Nechci vratit neco o tom ze jako cislo?
        # method to apply damage on this entity
        # rock takes damage only when it is hit with damage_amount of at least 10,
        # each such hit takes him 1 hitpoint, smaller hits don't cause any damage to him
        print(f"Auvi auvi, taking overall {damage_amount} damage.")
        self.hit_points = self.hit_points - damage_amount
        
        return self.hit_points

class LivingEntity(Entity):

    def __init__(self, name: str, coords: Tuple[int, int], hit_points: int, level: int, damage: int, attack_range: int) -> None:
        self.level = level
        self.damage = damage
        self.attack_range = attack_range
        super().__init__(name, coords, hit_points)

    def level_up(self) -> None:
        
        self.level = self.level + 1
        
        return self.level

    def take_damage(self, damage_amount: int) -> None:
        # method to apply damage on this entity
        # the whole damage_amount is applied to hitpoints

        print(f"Auvi auvi, taking overall {damage_amount} damage.")
        self.hit_points = self.hit_points - damage_amount
        
        return self.hit_points

    def hit(self, other_entity: Entity) -> None:
        # method to hit other entity to cause them to take damage
        # this is abstract method, just keep it as it is
        raise NotImplementedError("This method is abstract. Please implement it")  # in the child, not here :-)

    def move(self, vector: Tuple[int, int]) -> None:
        # method to make entity move on the board
        # it expects vector of steps in x and y directions, the vector is represented as tuple (x, y)
        # the move is executed by editing coordinates by the vector elements
        # x is editing x axis and y is editing y axis
        # +x, -x, +y, -y define also direction of move

        self.coords = (self.coords[0] + vector[0], self.coords[1] + vector[1])

        return self.coords

    def in_range(self, other_entity: Entity) -> bool:
        # method to check if other entity is in range
        # return True if get_distance to other entity <= self.range
        # return False if get_distance to other entity > self.range
        distance = super().get_distance(other_entity.coords)

        print(f"Hey kids, my name is: {self.name}, others name is {other_entity.name} and distsance between us is: {distance}")
        print(f"My range is: {self.attack_range}")

        if distance <= self.attack_range:

            print(f"Yeyy! I am in range to attack You :)")
            return True
        else:
            print("Cant attack You right now :(")
            return False

class Warrior(LivingEntity):

    def __init__(self, name, coords, hit_points, level, damage, attack_range):

        super().__init__(name, coords, hit_points, level, damage, attack_range)

    def hit(self, other_entity: Entity) -> None:
        # implement hit to another entity
        #   if other entity is in range make it to take damage with it's own method take_damage
        #       damage_amount applied to other entity is equal to damage of attacking entity
        if self.in_range(other_entity):
            other_entity.take_damage(self.damage)

    def take_damage(self, damage_amount: int) -> None:
        # method to apply damage on this entity
        # Warrior is taking damage only when the damage_amount is bigger then his level
        #   If damage is bigger than level Warrior takes the difference as a damage_amount
        #   Else Warrior takes no damage

        if damage_amount > self.level:
            diff = damage_amount - self.level
            self.hit_points = self.hit_points - diff
            print(f"Damn taking damage of {diff} damage_points ...")

            return self.hit_points 
        else:
            print("Haha cant hurt me!!")
            return self.hit_points

    def level_up(self) -> None:
        # this method should increase the level according to parent; use call to parent's method
        # also increase damage by 1 and hit_points by 2
        print(f'My name is {self.name} and I am leveling up and getting pretty OP')

        self.level = self.level + 1
        self.damage = self.damage + 1
        self.hit_points = self.hit_points + 2

        return self.level, self.damage, self.hit_points

class Archer(LivingEntity):
    
    def __init__(self, name, coords, hit_points, level, damage, attack_range):
        super().__init__(name, coords, hit_points, level, damage, attack_range)

    def take_damage(self, damage_amount: int) -> None:
        # method to apply damage on this entity                                                                         Because ctrl + c && ctrl + v works just fine :) 
        # Warrior is taking damage only when the damage_amount is bigger then his level
        #   If damage is bigger than level Warrior takes the difference as a damage_amount
        #   Else Warrior takes no damage
        self.hit_points -= damage_amount
        
        return self.hit_points

    def hit(self, other_entity: Entity) -> None:
        # implement hit to another entity
        #   if other entity is in range make it to take damage with it's own method take_damage
        #       damage_amount applied to other entity is equal to damage of attacking entity
        if self.in_range(other_entity):
            print(self.in_range(other_entity))
            res = other_entity.take_damage(self.damage)

            print(res)

            return res

    def level_up(self) -> None:
        # this method should increase the level according to parent; use call to parent's method
        # also increase damage by 2 and hit_points by 1
        self.damage += 2
        self.hit_points += 1
        self.level += 1

        
        return self.damage, self.hit_points, self.level



In [36]:
def test_this_thing():
    # Simple Entity tests
    entity = Entity("E", (0, 0), 10)
    # constructor
    assert entity.name == "E"
    assert entity.coords == (0, 0)
    assert entity.hit_points == 10
    # is alive check
    assert entity.is_alive()
    # get distance check
    assert entity.get_distance((0, 0)) == 0

    # Rock tests
    rocky = Rock("Rocky", (0, 1), 5)
    # constructor
    assert rocky.name == "Rocky"
    assert rocky.coords == (0, 1)
    assert rocky.hit_points == 5
    # is alive check
    assert rocky.is_alive()
    # get distance check
    assert rocky.get_distance((0, 0)) == 1
    # damage taking
    rocky.take_damage(7)
    assert rocky.hit_points == 5
    rocky.take_damage(100)
    assert rocky.hit_points == 4

    # Table tests
    table = Furniture("Table", (1, 0), 3)
    # constructor
    assert table.name == "Table"
    assert table.coords == (1, 0)
    assert table.hit_points == 3
    # is alive check
    assert table.is_alive()
    # get distance check
    assert table.get_distance((0, 0)) == 1
    # damage taking
    table.take_damage(2)
    assert table.hit_points == 1
    table.take_damage(2)
    assert table.hit_points == -1
    assert not table.is_alive()

    # LivingEntity tests
    tim = LivingEntity("Tim", (-1, 0), 3, 1, 1, 1)
    # constructor
    assert tim.name == "Tim"
    assert tim.coords == (-1, 0)
    assert tim.hit_points == 3
    assert tim.level == 1
    assert tim.damage == 1
    assert tim.attack_range == 1
    # is alive check
    assert tim.is_alive()
    # get distance check
    assert tim.get_distance((0, 0)) == 1
    # level up check
    tim.level_up()
    assert tim.level == 2
    # damage taking
    tim.take_damage(1)
    assert tim.hit_points == 2
    assert tim.is_alive()
    # move check
    assert tim.in_range(entity)
    tim.move((0, -1))
    assert not tim.in_range(entity)

    # Warrior tests
    willy = Warrior("William Wallace", (1, 1), 3, 5, 8, 2)
    # constructor
    assert willy.name == "William Wallace"
    assert willy.coords == (1, 1)
    assert willy.hit_points == 3
    assert willy.level == 5
    assert willy.damage == 8
    assert willy.attack_range == 2
    # is alive check
    assert willy.is_alive()
    # get distance check
    assert willy.get_distance((0, 0)) == 2
    # level up check
    willy.level_up()
    assert willy.hit_points == 5
    assert willy.level == 6
    assert willy.damage == 9
    # damage taking
    willy.take_damage(1)
    assert willy.hit_points == 5
    willy.take_damage(7)
    assert willy.hit_points == 4
    # fight checks
    rocky = Rock("Rocky", (0, 1), 5)
    willy.hit(rocky)
    assert rocky.hit_points == 5
    table = Furniture("Table", (1, 0), 3)
    willy.hit(table)
    assert table.hit_points == -6
    tim = LivingEntity("Tim", (-1, 0), 3, 1, 1, 1)
    willy.hit(tim)
    assert tim.hit_points == 3
    willy.move((-1, -1))
    assert willy.get_distance((0, 0)) == 0
    willy.hit(tim)
    assert tim.hit_points == -6

    # Archer tests
    rob = Archer("Robin Hood", (5, 8), 3, 1, 7, 10)
    # constructor
    assert rob.name == "Robin Hood"
    assert rob.coords == (5, 8)
    assert rob.hit_points == 3
    assert rob.level == 1
    assert rob.damage == 7
    assert rob.attack_range == 10
    # is alive check
    assert rob.is_alive()
    # get distance check
    assert rob.get_distance((0, 0)) == 13
    # level up check
    rob.level_up()
    assert rob.hit_points == 4
    assert rob.level == 2
    assert rob.damage == 9
    # damage taking
    rob.take_damage(1)
    assert rob.hit_points == 3
    # fight checks
    willy = Warrior("William Wallace", (1, 1), 3, 5, 8, 2)
    rob.hit(willy)
    assert willy.hit_points == 3
    rob.move((0, -1))
    rob.hit(willy)
    assert willy.hit_points == -1
    assert not willy.is_alive()

In [37]:
#test_this_thing()

In [38]:
# Simple Entity tests
entity = Entity("E", (0, 0), 10)
# constructor
assert entity.name == "E"
assert entity.coords == (0, 0)
assert entity.hit_points == 10
# is alive check
assert entity.is_alive()
# get distance check
assert entity.get_distance((0, 0)) == 0

# Rock tests
rocky = Rock("Rocky", (0, 1), 5)
# constructor
assert rocky.name == "Rocky"
assert rocky.coords == (0, 1)
assert rocky.hit_points == 5
# is alive check
assert rocky.is_alive()
# get distance check
assert rocky.get_distance((0, 0)) == 1
# damage taking
rocky.take_damage(7)
assert rocky.hit_points == 5
rocky.take_damage(100)
assert rocky.hit_points == 4

# Table tests
table = Furniture("Table", (1, 0), 3)
# constructor
assert table.name == "Table"
assert table.coords == (1, 0)
assert table.hit_points == 3
# is alive check
assert table.is_alive()
# get distance check
assert table.get_distance((0, 0)) == 1
# damage taking
table.take_damage(2)
assert table.hit_points == 1
table.take_damage(2)
assert table.hit_points == -1
assert not table.is_alive()

# LivingEntity tests
tim = LivingEntity("Tim", (-1, 0), 3, 1, 1, 1)
# constructor
assert tim.name == "Tim"
assert tim.coords == (-1, 0)
assert tim.hit_points == 3
assert tim.level == 1
assert tim.damage == 1
assert tim.attack_range == 1
# is alive check
assert tim.is_alive()
# get distance check
assert tim.get_distance((0, 0)) == 1
# level up check
tim.level_up()
assert tim.level == 2
# damage taking
tim.take_damage(1)
assert tim.hit_points == 2
assert tim.is_alive()
# move check
assert tim.in_range(entity)
tim.move((0, -1))
assert not tim.in_range(entity)

# Warrior tests
willy = Warrior("William Wallace", (1, 1), 3, 5, 8, 2)
# constructor
assert willy.name == "William Wallace"
assert willy.coords == (1, 1)
assert willy.hit_points == 3
assert willy.level == 5
assert willy.damage == 8
assert willy.attack_range == 2
# is alive check
assert willy.is_alive()
# get distance check
assert willy.get_distance((0, 0)) == 2
# level up check
willy.level_up()
assert willy.hit_points == 5
assert willy.level == 6
assert willy.damage == 9
# damage taking
willy.take_damage(1)
assert willy.hit_points == 5
willy.take_damage(7)
assert willy.hit_points == 4
# fight checks
rocky = Rock("Rocky", (0, 1), 5)
willy.hit(rocky)
assert rocky.hit_points == 5
table = Furniture("Table", (1, 0), 3)
willy.hit(table)
assert table.hit_points == -6
tim = LivingEntity("Tim", (-1, 0), 3, 1, 1, 1)
willy.hit(tim)
assert tim.hit_points == 3
willy.move((-1, -1))
assert willy.get_distance((0, 0)) == 0
willy.hit(tim)
assert tim.hit_points == -6

# Archer tests
rob = Archer("Robin Hood", (5, 8), 3, 1, 7, 10)
# constructor
assert rob.name == "Robin Hood"
assert rob.coords == (5, 8)
assert rob.hit_points == 3
assert rob.level == 1
assert rob.damage == 7
assert rob.attack_range == 10
# is alive check
assert rob.is_alive()
# get distance check
assert rob.get_distance((0, 0)) == 13
# level up check
rob.level_up()
assert rob.hit_points == 4
assert rob.level == 2
assert rob.damage == 9
# damage taking
rob.take_damage(1)
assert rob.hit_points == 3
# fight checks
willy = Warrior("William Wallace", (1, 1), 3, 5, 8, 2)
rob.hit(willy)
assert willy.hit_points == 3
rob.move((0, -1))
rob.hit(willy)
assert willy.hit_points == -1
assert not willy.is_alive()

Ziju, sem v cajku
Ziju, sem v cajku
Pohoda :)
 Auvi Auvi, ubral jsi mi jeden zivot :(, 
 ted mam kvuli tobe 4 zivotu
Ziju, sem v cajku
Auvi auvi, taking overall 2 damage.
Auvi auvi, taking overall 2 damage.
Ziju, sem v cajku
Auvi auvi, taking overall 1 damage.
Ziju, sem v cajku
Hey kids, my name is: Tim, others name is E and distsance between us is: 1
My range is: 1
Yeyy! I am in range to attack You :)
Hey kids, my name is: Tim, others name is E and distsance between us is: 2
My range is: 1
Cant attack You right now :(
Ziju, sem v cajku
My name is William Wallace and I am leveling up and getting pretty OP
Haha cant hurt me!!
Damn taking damage of 1 damage_points ...
Hey kids, my name is: William Wallace, others name is Rocky and distsance between us is: 1
My range is: 2
Yeyy! I am in range to attack You :)
Pohoda :)
Hey kids, my name is: William Wallace, others name is Table and distsance between us is: 1
My range is: 2
Yeyy! I am in range to attack You :)
Auvi auvi, taking overall 9 d